# The TinyALU TLM Testbench

In [1]:
from pyuvm import *
from tb_pkg import *
import random
import time

class AluSeqItem(uvm_sequence_item):

    def __init__(self, name, A=0, B=0, op=Ops.ADD):
        super().__init__(name)
        self.A = A
        self.B = B
        self.op = Ops(op)

    def __eq__(self, other):
        same = self.A == other.A \
               and self.B == other.B \
               and self.op == other.op
        return same

    def __str__(self):
        return f"{self.get_name()} : A: 0x{self.A:02x} OP: {self.op.name} ({self.op.value}) B: 0x{self.B:02x}"
    
    def randomize(self):
        self.A = random.randint(0,255)
        self.B = random.randint(0,255)
        self.op = random.choice(list(Ops))
        
class AluSeq(uvm_sequence):
    def body(self):
        for ii in range(5):
            cmd_tr = AluSeqItem("cmd_tr")
            self.start_item(cmd_tr) # UVM start_item gets the sequencer
            cmd_tr.randomize()
            self.finish_item(cmd_tr) # UVM finish_item waits for the driver to call item_done    

class Driver(uvm_driver):
    def build_phase(self):
        self.bfm = self.cdb_get("BFM")

    def run_phase(self):
        while True:
            command = self.seq_item_port.get_next_item()
            self.bfm.send_op(command)
            self.seq_item_port.item_done()
            
class Coverage(uvm_subscriber):
    
    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def write(self, cmd):
        self.cvg.add(cmd.op)

    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")

class Scoreboard(uvm_component):  

    def build_phase(self):
        self.cmd_fifo = uvm_tlm_analysis_fifo("cmd_fifo", self)
        self.rslt_fifo = uvm_tlm_analysis_fifo("rslt_fifo", self)
        self.cmd_get_port = uvm_get_port("cmd_get_port", self)    
        self.rslt_get_port = uvm_get_port("rslt_get_port", self) 
        self.cmd_export = self.cmd_fifo.analysis_export
        self.rslt_export = self.rslt_fifo.analysis_export

    def connect_phase(self):
        self.cmd_get_port.connect(self.cmd_fifo.get_export)
        self.rslt_get_port.connect(self.rslt_fifo.get_export)

    def run_phase(self):
        while True:
            rtxn = self.rslt_get_port.get()
            cmd = self.cmd_get_port.get()  
            actual_result = rtxn.result
            predicted_result = TinyAluTlm.alu_op(cmd.A, cmd.B, cmd.op)
            if predicted_result == actual_result:
                self.logger.info(f"PASSED: 0x{cmd.A:02x} {cmd.op.name} 0x{cmd.B:02x} ="
                                 f" 0x{actual_result:04x}")
            else:
                self.logger.error(f"FAILED: 0x{cmd.A:02x} {cmd.op.name} 0x{cmd.B:02x} "
                                  f"= 0x{actual_result:04x} expected 0x{predicted_result:04x}")     
                
class Monitor(uvm_component):
    def __init__(self, name, parent, method_name):
        super().__init__(name, parent)
        self.method_name = method_name
    
    def build_phase(self):
        self.dut = self.cdb_get("BFM")
        self.ap = uvm_analysis_port("ap", self)

    def run_phase(self):
        while True:
            get_method = getattr(self.dut, self.method_name)
            datum = get_method()
            self.ap.write(datum)    
            
class AluEnv(uvm_env):

    def build_phase(self):
        self.cmd_mon = Monitor("cmd_mon", self, "get_cmd")
        self.rslt_mon = Monitor("rslt_mon", self, "get_result")
        self.scoreboard = Scoreboard("scoreboard", self)
        self.coverage = Coverage("coverage", self)
        self.driver = Driver("driver", self)
        self.seqr  = uvm_sequencer("seqr", self)
        ConfigDB().set(None, "*", "SEQR", self.seqr)
        
    def connect_phase(self):
        self.cmd_mon.ap.connect(self.scoreboard.cmd_export)
        self.cmd_mon.ap.connect(self.coverage)
        self.rslt_mon.ap.connect(self.scoreboard.rslt_export)
        self.driver.seq_item_port.connect(self.seqr.seq_item_export)
        

class AluTest(uvm_test):
    def build_phase(self):
        self.dut = TinyAluTlm("BFM", self)
        self.cdb_set("BFM", self.dut)
        self.env = AluEnv("env", self)
    def run_phase(self):
        self.raise_objection()
        seqr = self.cdb_get("SEQR")
        seq = AluSeq("seq")
        seq.start(seqr)
        time.sleep(1)
        self.drop_objection()

uvm_root().run_test("AluTest")
        

INFO: <ipython-input-1-6c1b0b0fa9e2>(79)[uvm_test_top.env.scoreboard]: PASSED: 0xee ADD 0xa3 = 0x0191
INFO: <ipython-input-1-6c1b0b0fa9e2>(79)[uvm_test_top.env.scoreboard]: PASSED: 0x2b MUL 0x0e = 0x025a
INFO: <ipython-input-1-6c1b0b0fa9e2>(79)[uvm_test_top.env.scoreboard]: PASSED: 0xf0 ADD 0xeb = 0x01db
INFO: <ipython-input-1-6c1b0b0fa9e2>(79)[uvm_test_top.env.scoreboard]: PASSED: 0x27 XOR 0xd5 = 0x00f2
INFO: <ipython-input-1-6c1b0b0fa9e2>(79)[uvm_test_top.env.scoreboard]: PASSED: 0xee AND 0x01 = 0x0000
ERROR: <ipython-input-1-6c1b0b0fa9e2>(56)[uvm_test_top.env.coverage]: Functional coverage error. Missed: {<Ops.NOP: 0>}
